# [E-04]Lyricist

### 데이터 읽어오기

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()   # 텍스트를 라인 단위로 끊어서 list 형태로 읽어온다
        raw_corpus.extend(raw)
        
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### 데이터 정제

In [2]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고, 양쪽 공백을 지운다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백은 하나의 공백으로 바꾼다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다
    sentence = sentence.strip() # 다시 양쪽 공백을 지운다
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가
    return sentence

In [3]:
# 정제된 문장을 모으자
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뛴다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    # 정제를 하고 담자
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [4]:
import tensorflow as tf

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=20000,
        filters=' ',
        oov_token="<unk>" # out-of-vocabulary, 사전에 없는 단어
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f0d53581100>


In [5]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [6]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


### 평가 데이터셋 분리

In [7]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                         tgt_input, 
                                                         test_size=0.2,
                                                         shuffle=True,
                                                         random_state=34)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 20000개와, 여기 포함되지 않은 0:<pad>를 포함하여 20001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### 인공지능 만들기

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 14
hidden_size = 2048
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [14]:
# 데이터셋에서 데이터 한 배치만 불러온다
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣는다
lyricist(src_sample)

<tf.Tensor: shape=(256, 14, 20001), dtype=float32, numpy=
array([[[-2.76420924e-05,  9.80101140e-06, -2.36052784e-07, ...,
          1.27791936e-05, -5.61628167e-06,  4.93496300e-05],
        [-3.96364849e-05, -1.12778062e-05,  3.74307820e-05, ...,
          7.82936604e-06, -2.54278871e-06,  5.43452479e-05],
        [-7.55908404e-05, -2.63962556e-05,  3.45137014e-05, ...,
         -5.91262906e-05, -1.31379193e-05,  3.34340875e-05],
        ...,
        [ 2.10106678e-04,  7.67657621e-05, -1.23540871e-04, ...,
          3.34077231e-06, -1.91692336e-04,  1.18814634e-04],
        [ 1.97335365e-04,  9.53276904e-05, -1.57885530e-04, ...,
          8.61708104e-05, -1.75743364e-04,  1.63033605e-04],
        [ 1.77810260e-04,  1.17334879e-04, -1.84628327e-04, ...,
          1.67903156e-04, -1.50453823e-04,  2.08974598e-04]],

       [[-2.76420924e-05,  9.80101140e-06, -2.36052784e-07, ...,
          1.27791936e-05, -5.61628167e-06,  4.93496300e-05],
        [-4.91710853e-05,  2.07569428e-05,  2

In [15]:
lyricist.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  280014    
_________________________________________________________________
lstm_3 (LSTM)                multiple                  16900096  
_________________________________________________________________
lstm_4 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense_1 (Dense)              multiple                  40982049  
Total params: 91,724,783
Trainable params: 91,724,783
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 387s 561ms/step - loss: 4.0279
Epoch 2/10
686/686 [==============================] - 389s 567ms/step - loss: 3.4289
Epoch 3/10
686/686 [==============================] - 391s 570ms/step - loss: 3.1708
Epoch 4/10
686/686 [==============================] - 391s 570ms/step - loss: 3.0066
Epoch 5/10
686/686 [==============================] - 391s 569ms/step - loss: 2.8625
Epoch 6/10
686/686 [==============================] - 392s 572ms/step - loss: 2.7259
Epoch 7/10
686/686 [==============================] - 392s 572ms/step - loss: 2.5896
Epoch 8/10
686/686 [==============================] - 392s 570ms/step - loss: 2.4556
Epoch 9/10
686/686 [==============================] - 392s 571ms/step - loss: 2.3264
Epoch 10/10
 97/686 [===>..........................] - ETA: 5:37 - loss: 2.1883

In [16]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = lyricist(test_tensor)
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        # 3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
    
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

In [19]:
generate_text(lyricist, tokenizer, init_sentence="<start> we are", max_len=20)

'<start> we are the eggmen <end> '

# 회고

처음 코드를 완성했을때 loss도 낮았고 작동도 잘됐다.  
그런데 모델이 바보같이 짧고 재미없는 문장만 만들었다.  
다시 코드를 검토해보니 정제된 문장을 모으는데 짧은 문장들만 모으도록 되어있었다.  
  
코드를 다시 작성하고 loss 를 줄이기 위해 여러가지 시도를 해보았다.
첫번째는 embeding을 256, hidensize를 1024로 설정하니 loss 가 2.31까지 떨어졌다.
두번째는 다른 것은 잘 모르겠지만 RNN을 추가해서 한번 더 읽도록 만들면 loss가 줄어들거라 생각했다.  
RNN을 하나 더 추가하고 hiddensize를 1024, 그리고 다른 사람 코드를 보니까 embeding은 shape에 맞추는 거 같아서 14로 맞추었다.  
그런데 학습시간만 늘어나고 loss가 낮아지는 비율이 줄어들었다.  
세번째는 RNN을 추가하는 것은 의미가 없는 거 같아 RNN을 2개로 다시 되돌리고 embening을 14, hiddensize를 늘려서 2048로 설정하였다.  
시간이 없어서 10번째까지 돌리지는 못했지만 2.1까지 줄어들것을 예상된다.